Krishna Sharma | AP22110010128

In [35]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [36]:
red = pd.read_csv('winequality-red.csv',delimiter=';')
red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [37]:
white = pd.read_csv('winequality-white.csv',delimiter=';')
white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [38]:
X_train, X_test, y_train, y_test = train_test_split(white.values[:,:-1],white.values[:,-1:],test_size=0.25,random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (3673, 11) (3673,)
Testing dataset shape: (1225, 11) (1225,)


In [39]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
sfs1 = sfs(clf,k_features=5,forward=True,floating=True,verbose=2,scoring='accuracy',cv=5)
sfs1 = sfs1.fit(X_train, y_train)


[2025-03-20 14:58:48] Features: 1/5 -- score: 0.49686407533040466
[2025-03-20 14:58:55] Features: 2/5 -- score: 0.5415251441175926
[2025-03-20 14:59:03] Features: 3/5 -- score: 0.6054985263860313
[2025-03-20 14:59:11] Features: 4/5 -- score: 0.6264605460712895
[2025-03-20 14:59:19] Features: 5/5 -- score: 0.6376192329792952

In [40]:
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[1, 2, 3, 7, 10]


In [ ]:
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train[:, feat_cols], y_train)

y_train_pred = clf.predict(X_train[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test[:, feat_cols]) 
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on selected features: 0.558
Testing accuracy on selected features: 0.513


In [42]:
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
print('Training accuracy on all features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print('Testing accuracy on all features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on all features: 0.566
Testing accuracy on all features: 0.509


In [43]:
X = white.iloc[:, :-1]
y = white.iloc[:, -1]

correlations = X.corrwith(y)
sorted_features = correlations.abs().sort_values(ascending=False)
print("Feature Correlations with Target:\n", sorted_features)

Feature Correlations with Target:
 alcohol                 0.435575
density                 0.307123
chlorides               0.209934
volatile acidity        0.194723
total sulfur dioxide    0.174737
fixed acidity           0.113663
pH                      0.099427
residual sugar          0.097577
sulphates               0.053678
citric acid             0.009209
free sulfur dioxide     0.008158
dtype: float64


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Training dataset shape: {X_train.shape}, {y_train.shape}")
print(f"Testing dataset shape: {X_test.shape}, {y_test.shape}")

Training dataset shape: (3673, 6), (3673,)
Testing dataset shape: (1225, 6), (1225,)


In [51]:
rf_model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
sfs = sfs(rf_model, k_features=5,forward=False,floating=True,scoring='accuracy',cv=5)    

In [54]:
sfs.fit(X_train, y_train)
selected_features = list(sfs.k_feature_idx_)
print("Selected Features:", X.columns[selected_features].tolist())

Selected Features: ['alcohol', 'density', 'chlorides', 'volatile acidity', 'total sulfur dioxide']


In [57]:
rf_model.fit(X_train.iloc[:, selected_features], y_train)

y_train_pred = rf_model.predict(X_train.iloc[:, selected_features])
y_test_pred = rf_model.predict(X_test.iloc[:, selected_features])

print(f"Training Accuracy (Selected Features): {accuracy_score(y_train, y_train_pred):.3f}")
print(f"Testing Accuracy (Selected Features): {accuracy_score(y_test, y_test_pred):.3f}")

Training Accuracy (Selected Features): 1.000
Testing Accuracy (Selected Features): 0.648


Krishna Sharma | AP22110010128